In [1]:
%%capture
! wget https://raw.githubusercontent.com/zadushevno/nlp-4-project/main/aspect_extraction/pred_aspects.txt
! wget https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/dev_reviews.txt

In [2]:
import pandas as pd
import csv

In [3]:
test_texts_path = 'dev_reviews.txt'
pred_aspects_path = 'pred_aspects.txt'

In [4]:
texts = pd.read_csv(test_texts_path, delimiter='\t', names = ['review_id', 'text'])
texts.head(5)

,review_id,text
0,13823,"Зашли в""аппетит"" случайно.Не смотря на то,что ..."
1,1427,Здравствуйте!Посетили ваше заведение вчера пер...
2,16714,"Были в пятницу (19.03.10), заказывали столик д..."
3,797,"Были в ресторане 2 раза. Один раз днем, все по..."
4,34710,Удивляюсь отзывам про хорошее обслуживание. Бы...


In [5]:
pred_aspects = pd.read_csv(pred_aspects_path, delimiter='\t', names = ['review_id', 'category', 'mention', 'start', 'stop'])
pred_aspects.head(5)

,review_id,category,mention,start,stop
0,13823,Whole,аппетит,8,15
1,13823,Service,встретил,138,146
2,13823,Service,менеджер,147,155
3,13823,Service,девушка,179,186
4,13823,Service,проводила к столу,188,205


In [6]:
data = pd.merge(texts, pred_aspects, on='review_id')
data.head(5)

,review_id,text,category,mention,start,stop
0,13823,"Зашли в""аппетит"" случайно.Не смотря на то,что ...",Whole,аппетит,8,15
1,13823,"Зашли в""аппетит"" случайно.Не смотря на то,что ...",Service,встретил,138,146
2,13823,"Зашли в""аппетит"" случайно.Не смотря на то,что ...",Service,менеджер,147,155
3,13823,"Зашли в""аппетит"" случайно.Не смотря на то,что ...",Service,девушка,179,186
4,13823,"Зашли в""аппетит"" случайно.Не смотря на то,что ...",Service,проводила к столу,188,205


In [7]:
data['totrain'] = '@ ' + data['mention'] + ' @ ' + data['text']
data.head(5)

,review_id,text,category,mention,start,stop,totrain
0,13823,"Зашли в""аппетит"" случайно.Не смотря на то,что ...",Whole,аппетит,8,15,"@ аппетит @ Зашли в""аппетит"" случайно.Не смотр..."
1,13823,"Зашли в""аппетит"" случайно.Не смотря на то,что ...",Service,встретил,138,146,"@ встретил @ Зашли в""аппетит"" случайно.Не смот..."
2,13823,"Зашли в""аппетит"" случайно.Не смотря на то,что ...",Service,менеджер,147,155,"@ менеджер @ Зашли в""аппетит"" случайно.Не смот..."
3,13823,"Зашли в""аппетит"" случайно.Не смотря на то,что ...",Service,девушка,179,186,"@ девушка @ Зашли в""аппетит"" случайно.Не смотр..."
4,13823,"Зашли в""аппетит"" случайно.Не смотря на то,что ...",Service,проводила к столу,188,205,"@ проводила к столу @ Зашли в""аппетит"" случайн..."


In [8]:
sentences = data.totrain.values

Чтобы скачать преобученную модель из [нашей папки](https://drive.google.com/drive/u/0/folders/1e8WjOU0U33uCS4XWpkWmb2F6G2aMXak2), нужно добавить ярлык папки к себе в гугл драйв и запустить код

In [9]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [10]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained('sberbank-ai/ruBert-base')
pred_model = AutoModelForSequenceClassification.from_pretrained('/content/gdrive/MyDrive/absa-project').to('cuda')

config.json:   0%|          | 0.00/590 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

In [11]:
from transformers import pipeline
from transformers import AutoConfig

In [12]:
pipe = pipeline('text-classification', model=pred_model, tokenizer=tokenizer, device='cuda')

In [13]:
mapping = {
    "LABEL_0":"neutral",
    "LABEL_1":"positive",
    "LABEL_2":"both",
    "LABEL_3":"negative"
}

In [14]:
labels = []
for sentence in sentences:
    labels.append(mapping[pipe(sentence)[0]['label']])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [15]:
data['labels'] = labels
data.head(5)

,review_id,text,category,mention,start,stop,totrain,labels
0,13823,"Зашли в""аппетит"" случайно.Не смотря на то,что ...",Whole,аппетит,8,15,"@ аппетит @ Зашли в""аппетит"" случайно.Не смотр...",neutral
1,13823,"Зашли в""аппетит"" случайно.Не смотря на то,что ...",Service,встретил,138,146,"@ встретил @ Зашли в""аппетит"" случайно.Не смот...",neutral
2,13823,"Зашли в""аппетит"" случайно.Не смотря на то,что ...",Service,менеджер,147,155,"@ менеджер @ Зашли в""аппетит"" случайно.Не смот...",neutral
3,13823,"Зашли в""аппетит"" случайно.Не смотря на то,что ...",Service,девушка,179,186,"@ девушка @ Зашли в""аппетит"" случайно.Не смотр...",neutral
4,13823,"Зашли в""аппетит"" случайно.Не смотря на то,что ...",Service,проводила к столу,188,205,"@ проводила к столу @ Зашли в""аппетит"" случайн...",neutral


In [18]:
data = data.drop(['text', 'totrain'], axis=1)
data.head(5)

,review_id,category,mention,start,stop,labels
0,13823,Whole,аппетит,8,15,neutral
1,13823,Service,встретил,138,146,neutral
2,13823,Service,менеджер,147,155,neutral
3,13823,Service,девушка,179,186,neutral
4,13823,Service,проводила к столу,188,205,neutral


In [19]:
data.to_csv('pred_aspects_with_sent.txt', sep='\t', index=False, header=False)